In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import random
from scipy.stats import norm, t, chi2

In [92]:
p = 0.95

In [93]:
def get_sums(matrix):
    sums_i = [sum(line) for line in matrix]
    sums_j = [sum([matrix[i][j] for i in range(len(matrix))]) for j in range(len(matrix[0]))]
    return sums_i, sums_j

In [94]:
def get_avgs(matrix):
    avgs_i = [sum(line)/len(line) for line in matrix]
    avgs_j = [sum([matrix[i][j] for i in range(len(matrix))])/len(matrix) for j in range(len(matrix[0]))]
    return avgs_i, avgs_j

In [95]:
# Выборочная дисперсия от фактора случайности
def D(matrix):
    k = len(matrix[0])
    m = len(matrix)
    avgs_hatch, avgs = get_avgs(matrix)
    matrix_avg = sum([sum(line) for line in matrix]) / (k*m)
    result = 0
    for i in range(k):
        for j in range(m):
            result += (matrix[j][i] - avgs[i]) ** 2
    for j in range(m):
        result -= k * ((avgs_hatch[j] - matrix_avg)**2)
    return result / ((k-1)*(m-1))

In [96]:
# Выборочная дисперсия, включающая влияние фактора А и дисперсию от фактора случайности
def DnA(matrix):
    k = len(matrix[0])
    m = len(matrix)
    _, avgs = get_avgs(matrix)
    matrix_avg = sum([sum(line) for line in matrix]) / (k*m)
    result = 0
    for i in range(k):
        result += (avgs[i] - matrix_avg) ** 2
    return m/(k-1) * result

In [97]:
# Выборочная дисперсия, включающая влияние фактора B и дисперсию от фактора случайности
def DnB(matrix):
    k = len(matrix[0])
    m = len(matrix)
    avgs_hatch, _ = get_avgs(matrix)
    matrix_avg = sum([sum(line) for line in matrix]) / (k*m)
    result = 0
    for j in range(m):
        result += (avgs_hatch[j] - matrix_avg) ** 2
    return k/(m-1) * result

In [98]:
#Оценка генеральной дисперсии если фактор А незначимый
def genS_A(matrix):
    k = len(matrix[0])
    m = len(matrix)
    return ((k-1)*DnA(matrix)+(k-1)*(m-1)*D(matrix))/((k-1)+(k-1)*(m-1))

In [99]:
#Оценка генеральной дисперсии если фактор B незначимый
def genS_B(matrix):
    k = len(matrix[0])
    m = len(matrix)
    return ((m-1)*DnB(matrix)+(k-1)*(m-1)*D(matrix))/((m-1)+(k-1)*(m-1))

In [100]:
#Оценка генеральной дисперсии если фактор A и фактор B незначимые
def genS(matrix):
    k = len(matrix[0])
    m = len(matrix)
    return ((k-1)*DnA(matrix)+(m-1)*DnB(matrix)+(k-1)*(m-1)*D(matrix))/((k-1)+(m-1)+(k-1)*(m-1))

In [101]:
#Оценка дисперсии фактора A
def SA(matrix):
    m = len(matrix)
    return (DnA(matrix) - D(matrix)) / m

In [102]:
#Оценка дисперсии фактора B
def SB(matrix):
    k = len(matrix[0])
    return (DnB(matrix) - D(matrix)) / k

In [125]:
matrix = [
    [3.83, 3, 2.7],
    [4.1, 3.13, 3.5],
    [3.63, 3.6, 3.2],
    [3.27, 3.5, 3.7]
]

matrix1 = [
    [3, 7, 3, 6, 6, 7, 6, 3, 8, 3],
    [6, 5, 7, 4, 9, 4, 3, 2, 7, 8],
    [8, 6, 3, 2, 7, 8, 6, 9, 3, 8],
    [4, 7, 7, 8, 6, 4, 5, 8, 4, 7],
    [6, 2, 6, 6, 8, 9, 7, 6, 8, 1]
]

avgs_i, avgs_j = get_avgs(matrix)
for i in range(len(matrix)+1):
    for j in range(len(matrix[0])+1):
        if i < len(matrix) and j < len(matrix[0]):
            print(f"{matrix[i][j]:.2f}", end="\t")
        elif i < len(matrix):
            print(f"{avgs_i[i]:.2f}", end="\t")
        elif j < len(matrix[0]):
            print(f"{avgs_j[j]:.2f}", end="\t")
    print()

3.83	3.00	2.70	3.18	
4.10	3.13	3.50	3.58	
3.63	3.60	3.20	3.48	
3.27	3.50	3.70	3.49	
3.71	3.31	3.28	


# Matrix 7

In [104]:
print("Выборочная дисперсия от фактора случайности:",D(matrix))

Выборочная дисперсия от фактора случайности: 0.15137500000000012


In [105]:
if DnA(matrix)/D(matrix) > sts.f.ppf(p, len(matrix[0])-1, (len(matrix[0])-1)*len(matrix)):
    a_important = True
    print("Фактор А значимый")
else:
    a_important = False
    print("Фактор А незначимый")

Фактор А незначимый


In [106]:
if DnB(matrix)/D(matrix) > sts.f.ppf(p, len(matrix[0])-1, (len(matrix[0])-1)*len(matrix)):
    b_important = True
    print("Фактор B значимый")
else:
    b_important = False
    print("Фактор B незначимый")

Фактор B незначимый


In [107]:
if not a_important:
    print("Оценка генеральной дисперсии учитывая что параметр А незначимый:", genS_A(matrix))
else:
    print("Оценка параметра А:", SA(matrix))

Оценка генеральной дисперсии учитывая что параметр А незначимый: 0.1715499999999999


In [108]:
if not b_important:
    print("Оценка генеральной дисперсии учитывая что параметр B незначимый:", genS_B(matrix))
else:
    print("Оценка параметра B:", SB(matrix))

Оценка генеральной дисперсии учитывая что параметр B незначимый: 0.13140555555555553


In [110]:
if not a_important and not b_important:
    print("Оценка генеральной дисперсии учитывая что параметры А и В незначимые:", genS(matrix))

Оценка генеральной дисперсии учитывая что параметры А и В незначимые: 0.14970909090909076


In [122]:
print("Фактор случайности:",D(matrix))
print("Выборочная дисперсия учитывающая влияние фактора А:", DnA(matrix))
print("Выборочная дисперсия учитывающая влияние фактора B:", DnB(matrix))

Фактор случайности: 0.15137500000000012
Выборочная дисперсия учитывающая влияние фактора А: 0.23207499999999925
Выборочная дисперсия учитывающая влияние фактора B: 0.09146666666666635


# Matrix 7_1

In [126]:
print("Выборочная дисперсия от фактора случайности:",D(matrix1))

Выборочная дисперсия от фактора случайности: 5.625555555555555


In [127]:
if DnA(matrix1)/D(matrix1) > sts.f.ppf(p, len(matrix1[0])-1, (len(matrix1[0])-1)*len(matrix1)):
    a_important = True
    print("Фактор А значимый")
else:
    a_important = False
    print("Фактор А незначимый")

Фактор А незначимый


In [128]:
if DnB(matrix1)/D(matrix1) > sts.f.ppf(p, len(matrix1[0])-1, (len(matrix1[0])-1)*len(matrix1)):
    b_important = True
    print("Фактор B значимый")
else:
    b_important = False
    print("Фактор B незначимый")

Фактор B незначимый


In [129]:
if not a_important:
    print("Оценка генеральной дисперсии учитывая что параметр А незначимый:", genS_A(matrix1))
else:
    print("Оценка параметра А:", SA(matrix1))

Оценка генеральной дисперсии учитывая что параметр А незначимый: 4.911111111111111


In [130]:
if not b_important:
    print("Оценка генеральной дисперсии учитывая что параметр B незначимый:", genS_B(matrix1))
else:
    print("Оценка параметра B:", SB(matrix1))

Оценка генеральной дисперсии учитывая что параметр B незначимый: 5.1899999999999995


In [131]:
if not a_important and not b_important:
    print("Оценка генеральной дисперсии учитывая что параметры А и В незначимые:", genS(matrix))

Оценка генеральной дисперсии учитывая что параметры А и В незначимые: 0.14970909090909076


In [132]:
print("Фактор случайности:",D(matrix1))
print("Выборочная дисперсия учитывающая влияние фактора А:", DnA(matrix1))
print("Выборочная дисперсия учитывающая влияние фактора B:", DnB(matrix1))

Фактор случайности: 5.625555555555555
Выборочная дисперсия учитывающая влияние фактора А: 2.0533333333333337
Выборочная дисперсия учитывающая влияние фактора B: 1.2699999999999998
